In [105]:
import pandas as pd
from PIL import Image
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf
from tensorflow.keras import layers, Model
import random
import tensorflow_addons as tfa

In [106]:
trainPath = "/kaggle/input/trainingpatientmetadataforrsnabreastcancer/train.csv"

train_df = pd.read_csv(trainPath)
print(train_df.info())

train_data = train_df.drop(["cancer"], axis=1, inplace=False)
train_labels = train_df["cancer"]

for i in range(300):
    X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels,
        test_size=0.2, shuffle = False, random_state = 45)
X_val = X_val.reset_index()
y_val = y_val.reset_index()

probTrain = y_train.value_counts()[1]/(y_train.value_counts()[0]+y_train.value_counts()[1])
probVal = len(y_val[y_val.cancer==1])/(len(y_val[y_val.cancer==1])+len(y_val[y_val.cancer==0]))

print(probTrain, probVal)

# print(len(y_train[y_train.cancer==1]))
# print(len(y_val[y_val.cancer==1]))
# X_val.index = np.arange(0, len(X_val))
# y_val.index = np.arange(0, len(y_val))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54706 entries, 0 to 54705
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   site_id                  54706 non-null  int64  
 1   patient_id               54706 non-null  int64  
 2   image_id                 54706 non-null  int64  
 3   laterality               54706 non-null  object 
 4   view                     54706 non-null  object 
 5   age                      54669 non-null  float64
 6   cancer                   54706 non-null  int64  
 7   biopsy                   54706 non-null  int64  
 8   invasive                 54706 non-null  int64  
 9   BIRADS                   26286 non-null  float64
 10  implant                  54706 non-null  int64  
 11  density                  29470 non-null  object 
 12  machine_id               54706 non-null  int64  
 13  difficult_negative_case  54706 non-null  bool   
dtypes: bool(1), float64(2)

In [107]:
imgBasePath = "/kaggle/input/rsna-breast-cancer-256-pngs/"
LCC_train = []
LMLO_train = []
RCC_train = []
RMLO_train = []

In [108]:
zero = 0
one = 0
for i in range(len(X_train)):
#     if(i == 100):
#       break
    rnd = random.random()
    laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
    label = y_train[i]
    pat_id = X_train._get_value(i, "patient_id")
    img_id = X_train._get_value(i, "image_id")
    imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
    #print(i, imgPath)
    #file = Path(imgPath)
    #img = Image.open(imgPath)
    #img = img.resize((256, 256))
    #imgArr = np.asarray(img)
    #imgArr = imgArr / 255
    
    if(laterality == "LCC"):
        if(label == 1 or rnd <= probTrain):
            LCC_train.append(imgPath)
            if(label == 0):
                zero+=1
            else:
                one+=1
    elif(laterality == "LMLO"):
        if(label == 1 or rnd <= probTrain):
            LMLO_train.append(imgPath)
            if(label == 0):
                zero+=1
            else:
                one+=1
    elif(laterality == "RCC"):
        if(label == 1 or rnd <= probTrain):
            RCC_train.append(imgPath)
            if(label == 0):
                zero+=1
            else:
                one+=1
    elif(laterality == "RMLO"):
        if(label == 1 or rnd <= probTrain):
            RMLO_train.append(imgPath)
            if(label == 0):
                zero+=1
            else:
                one+=1
    #print(len(X_train)-i, end=" ")

In [109]:
print(one/(zero+one))

0.49186550976138826


In [110]:
imgBasePath = "/kaggle/input/rsna-breast-cancer-256-pngs/"
LCC_val = []
LMLO_val = []
RCC_val = []
RMLO_val = []

In [111]:
print(X_val.head())

   index  site_id  patient_id    image_id laterality view   age  biopsy  \
0  43764        2       57164  2063547844          L  MLO  50.0       1   
1  43765        2       57164  1522121544          R   CC  50.0       0   
2  43766        2       57164  2060537563          R  MLO  50.0       0   
3  43767        1        5717   544564406          L   CC  59.0       0   
4  43768        1        5717  1031134031          L  MLO  59.0       0   

   invasive  BIRADS  implant density  machine_id  difficult_negative_case  
0         0     NaN        0     NaN          48                     True  
1         0     NaN        0     NaN          48                    False  
2         0     NaN        0     NaN          48                    False  
3         0     1.0        0       B          93                    False  
4         0     1.0        0       B          93                    False  


In [112]:
for i in range(len(X_val)):
#     if(i == 100):
#       break
    label = int(y_val.at[i, "cancer"])
    rnd = random.random()
    laterality = X_val._get_value(i, "laterality") + X_val._get_value(i, "view")
    # label = y_train[i]
    pat_id = X_val._get_value(i, "patient_id")
    img_id = X_val._get_value(i, "image_id")
    imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
    #print(i, imgPath)
    #file = Path(imgPath)
    #img = Image.open(imgPath)
    #img = img.resize((256, 256))
    #imgArr = np.asarray(img)
    #imgArr = imgArr / 255
    if(laterality == "LCC"):
        #if(label == 1 or rnd <= probVal):
        LCC_val.append(imgPath)
        if(label == 0):
            zero+=1
        else:
            one+=1
    elif(laterality == "LMLO"):
        #if(label == 1 or rnd <= probVal):
        LMLO_val.append(imgPath)
        if(label == 0):
            zero+=1
        else:
            one+=1
    elif(laterality == "RCC"):
        #if(label == 1 or rnd <= probVal):
        RCC_val.append(imgPath)
        if(label == 0):
            zero+=1
        else:
            one+=1
    elif(laterality == "RMLO"):
        #if(label == 1 or rnd <= probVal):
        RMLO_val.append(imgPath)
        if(label == 0):
            zero+=1
        else:
            one+=1
    #print(len(X_val)-i, end=" ")

In [113]:
print(one/(zero+one))

0.09043260580458422


In [114]:
print(len(X_val))
print(np.count_nonzero(RMLO_val==0))

10942
0


In [115]:
import random
def generator(filenames, X_dataset, y_dataset):
    def callable_generator():
        for filename in filenames:
            f = filename
            rf = f[len(imgBasePath):-4].split("_")
            pat_id = rf[0]
            img_id = rf[1]
            arr = X_dataset["image_id"].to_numpy()
            i = X_dataset.index[X_dataset['image_id'] == int(img_id)].tolist()
            label = float(y_dataset._get_value(i[0], "cancer"))
#             label = y_dataset.iloc(i)["cancer"]
#             print(label)
#             #print(type(label))
            img = Image.open(f)
            imgArr = np.asarray(img)
            imgArr = imgArr / 255
            imgArr = np.reshape(imgArr, (256, 256))
            yield imgArr, label
    return callable_generator

In [116]:
LCC_train_dataset = tf.data.Dataset.from_generator(generator(LCC_train, X_train, y_train), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
LMLO_train_dataset = tf.data.Dataset.from_generator(generator(LMLO_train, X_train, y_train), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
RCC_train_dataset = tf.data.Dataset.from_generator(generator(RCC_train, X_train, y_train), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
RMLO_train_dataset = tf.data.Dataset.from_generator(generator(RMLO_train, X_train, y_train), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))


In [117]:
LCC_val_dataset = tf.data.Dataset.from_generator(generator(LCC_val, X_val, y_val), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
LMLO_val_dataset = tf.data.Dataset.from_generator(generator(LMLO_val, X_val, y_val), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
RCC_val_dataset = tf.data.Dataset.from_generator(generator(RCC_val, X_val, y_val), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))
RMLO_val_dataset = tf.data.Dataset.from_generator(generator(RMLO_val, X_val, y_val), output_types=(tf.float32, tf.float32), output_shapes=([256, 256], []))

In [118]:
BATCH_SIZE = 16
def prep_data(train_dataset):
    train_dataset = train_dataset.cache().batch(BATCH_SIZE)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return train_dataset

In [119]:
LCC_train_dataset = prep_data(LCC_train_dataset)
LMLO_train_dataset = prep_data(LMLO_train_dataset)
RCC_train_dataset = prep_data(RCC_train_dataset)
RMLO_train_dataset = prep_data(RMLO_train_dataset)

In [120]:
# for lcc_batch in LCC_train_dataset:
#    print(lcc_batch[0])
# print("\n-------------------------------------------------------\n")

# for lcc_batch in LCC_val_dataset:
#    print(lcc_batch[0])

In [121]:
LCC_val_dataset = prep_data(LCC_val_dataset)
LMLO_val_dataset = prep_data(LMLO_val_dataset)
RCC_val_dataset = prep_data(RCC_val_dataset)
RMLO_val_dataset = prep_data(RMLO_val_dataset)

In [122]:
def convBlock(num_filters=[128,64],kernel_size=[7,5],stride=2):
    def customLayer(inputs):
        x = layers.Conv2D(num_filters[0],kernel_size[0],activation='relu')(inputs)
        x = layers.Conv2D(num_filters[1],kernel_size[1],activation='relu')(x)
        x = layers.MaxPooling2D(stride)(x)
        x = layers.BatchNormalization()(x)
        return x
    return customLayer

In [123]:
def cnnModel(input_shape,num_classes):
    inputs = layers.Input(input_shape)
    x = convBlock(num_filters=[128,64],kernel_size=[7,5],stride=2)(inputs)
    x = layers.Dropout(0.4)(x)
    x = convBlock(num_filters=[64,32],kernel_size=[5,3],stride=2)(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512,activation='relu')(x)
    x = layers.Dense(128,activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(num_classes,activation='softmax')(x)
    return Model(inputs=[inputs],outputs=[output])

In [124]:
EPOCHS=25
early_stopping=tf.keras.callbacks.EarlyStopping(patience=7,restore_best_weights=True)

In [125]:
# rm -rf /kaggle/working/

In [126]:
def pfbeta(labels, predictions, beta=1):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

In [129]:
model = cnnModel((256, 256, 1),num_classes=1)
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.50), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
LCC_model = model.fit(LCC_train_dataset, epochs=EPOCHS, validation_data=LCC_val_dataset, callbacks=[early_stopping])
hist_df = pd.DataFrame(LCC_model.history) 
hist_df.to_csv("LCC_model_V1_hist.csv")
model.save("LCC_model_V1.h5")

Epoch 1/25


2023-03-23 01:53:14.095685: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_14/dropout_42/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


28/28 [==============================] - 19s 603ms/step - loss: 12.4541 - f1_score: 0.6746 - precision_3: 0.5089 - recall_3: 1.0000 - val_loss: 101.7334 - val_f1_score: 0.0426 - val_precision_3: 0.0218 - val_recall_3: 1.0000
Epoch 2/25
28/28 [==============================] - 16s 569ms/step - loss: 8.7895 - f1_score: 0.6746 - precision_3: 0.5089 - recall_3: 1.0000 - val_loss: 29.2243 - val_f1_score: 0.0426 - val_precision_3: 0.0218 - val_recall_3: 1.0000
Epoch 3/25
28/28 [==============================] - 16s 573ms/step - loss: 2.2329 - f1_score: 0.6746 - precision_3: 0.5089 - recall_3: 1.0000 - val_loss: 8.4123 - val_f1_score: 0.0426 - val_precision_3: 0.0218 - val_recall_3: 1.0000
Epoch 4/25
28/28 [==============================] - 16s 571ms/step - loss: 1.1524 - f1_score: 0.6746 - precision_3: 0.5089 - recall_3: 1.0000 - val_loss: 2.3500 - val_f1_score: 0.0426 - val_precision_3: 0.0218 - val_recall_3: 1.0000
Epoch 5/25
28/28 [==============================] - 16s 575ms/step - loss: 

In [130]:
model = cnnModel((256, 256, 1),num_classes=1)
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.50), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
LMLO_model = model.fit(LMLO_train_dataset, epochs=EPOCHS, validation_data=LMLO_val_dataset, callbacks=[early_stopping])
hist_df = pd.DataFrame(LMLO_model.history) 
hist_df.to_csv("LMLO_model_V1_hist.csv")
model.save("LMLO_model_V1.h5")

Epoch 1/25


2023-03-23 02:04:02.062214: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_15/dropout_45/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


31/31 [==============================] - 42s 1s/step - loss: 14.7790 - f1_score: 0.6566 - precision_4: 0.4888 - recall_4: 1.0000 - val_loss: 7.8875 - val_f1_score: 0.0441 - val_precision_4: 0.0226 - val_recall_4: 1.0000
Epoch 2/25
31/31 [==============================] - 25s 831ms/step - loss: 5.4748 - f1_score: 0.6566 - precision_4: 0.4888 - recall_4: 1.0000 - val_loss: 5.7549 - val_f1_score: 0.0441 - val_precision_4: 0.0226 - val_recall_4: 1.0000
Epoch 3/25
31/31 [==============================] - 26s 831ms/step - loss: 3.1127 - f1_score: 0.6566 - precision_4: 0.4888 - recall_4: 1.0000 - val_loss: 5.0860 - val_f1_score: 0.0441 - val_precision_4: 0.0226 - val_recall_4: 1.0000
Epoch 4/25
31/31 [==============================] - 25s 827ms/step - loss: 1.2736 - f1_score: 0.6566 - precision_4: 0.4888 - recall_4: 1.0000 - val_loss: 8.2769 - val_f1_score: 0.0441 - val_precision_4: 0.0226 - val_recall_4: 1.0000
Epoch 5/25
31/31 [==============================] - 26s 844ms/step - loss: 1.2084

In [132]:
model = cnnModel((256, 256, 1),num_classes=1)
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.50), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
RCC_model = model.fit(RCC_train_dataset, epochs=EPOCHS, validation_data=RCC_val_dataset, callbacks=[early_stopping])
hist_df = pd.DataFrame(RCC_model.history) 
hist_df.to_csv("RCC_model_V1_hist.csv")
model.save("RCC_model_V1.h5")

Epoch 1/25


2023-03-23 02:17:56.945057: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_17/dropout_51/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


28/28 [==============================] - 31s 1s/step - loss: 13.1235 - f1_score: 0.6667 - precision_6: 0.5000 - recall_6: 1.0000 - val_loss: 0.7453 - val_f1_score: 0.0447 - val_precision_6: 0.0229 - val_recall_6: 1.0000
Epoch 2/25
28/28 [==============================] - 15s 563ms/step - loss: 8.1762 - f1_score: 0.6667 - precision_6: 0.5000 - recall_6: 1.0000 - val_loss: 1.2708 - val_f1_score: 0.0447 - val_precision_6: 0.0229 - val_recall_6: 1.0000
Epoch 3/25
28/28 [==============================] - 16s 565ms/step - loss: 3.7961 - f1_score: 0.6667 - precision_6: 0.5000 - recall_6: 1.0000 - val_loss: 2.5917 - val_f1_score: 0.0447 - val_precision_6: 0.0229 - val_recall_6: 1.0000
Epoch 4/25
28/28 [==============================] - 16s 566ms/step - loss: 1.2255 - f1_score: 0.6667 - precision_6: 0.5000 - recall_6: 1.0000 - val_loss: 0.7754 - val_f1_score: 0.0447 - val_precision_6: 0.0229 - val_recall_6: 1.0000
Epoch 5/25
28/28 [==============================] - 17s 629ms/step - loss: 0.8503

In [131]:
model = cnnModel((256, 256, 1),num_classes=1)
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.50), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
RMLO_model = model.fit(RMLO_train_dataset, epochs=EPOCHS, validation_data=RMLO_val_dataset, callbacks=[early_stopping])
hist_df = pd.DataFrame(RMLO_model.history) 
hist_df.to_csv("RMLO_model_V1_hist.csv")
model.save("RMLO_model_V1.h5")

Epoch 1/25


2023-03-23 02:11:21.415280: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_16/dropout_48/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


30/30 [==============================] - 40s 1s/step - loss: 11.8266 - f1_score: 0.6407 - precision_5: 0.4713 - recall_5: 1.0000 - val_loss: 1.1073 - val_f1_score: 0.0465 - val_precision_5: 0.0238 - val_recall_5: 1.0000
Epoch 2/25
30/30 [==============================] - 25s 852ms/step - loss: 5.4847 - f1_score: 0.6407 - precision_5: 0.4713 - recall_5: 1.0000 - val_loss: 2.0526 - val_f1_score: 0.0465 - val_precision_5: 0.0238 - val_recall_5: 1.0000
Epoch 3/25
30/30 [==============================] - 26s 868ms/step - loss: 2.5067 - f1_score: 0.6407 - precision_5: 0.4713 - recall_5: 1.0000 - val_loss: 0.6613 - val_f1_score: 0.0465 - val_precision_5: 0.0238 - val_recall_5: 1.0000
Epoch 4/25
30/30 [==============================] - 26s 867ms/step - loss: 1.6859 - f1_score: 0.6407 - precision_5: 0.4713 - recall_5: 1.0000 - val_loss: 0.5924 - val_f1_score: 0.0465 - val_precision_5: 0.0238 - val_recall_5: 1.0000
Epoch 5/25
30/30 [==============================] - 25s 845ms/step - loss: 0.9308

In [28]:
# # LCC_model = model.fit(LCC_train_dataset, epochs=EPOCHS, validation_data=LCC_val_dataset, callbacks=[early_stopping], steps_per_epoch=250, validation_steps=250)
# LCC_model = model.fit(LCC_train_dataset, epochs=EPOCHS, validation_data=LCC_val_dataset)
# hist_df = pd.DataFrame(LCC_model.history) 
# hist_df.to_csv("LCC_model_V1_hist.csv")
# model.save("LCC_model_V1.h5")

In [29]:
# LCC_model = model.fit(LCC_train_dataset, epochs=EPOCHS, validation_data=LCC_val_dataset)

In [30]:
# rm -rf /kaggle/working/

In [31]:
# for i in range(len(X_train)):
#     if(i == 100):
#       break
    
#     laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#     label = y_train[i]
#     pat_id = X_train._get_value(i, "patient_id")
#     img_id = X_train._get_value(i, "image_id")
#     imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#     #print(i, imgPath)
#     #file = Path(imgPath)
#     img = Image.open(imgPath)
#     #img = img.resize((256, 256))
#     imgArr = np.asarray(img)
#     imgArr = imgArr / 255
#     if(laterality == "LCC"):
#         LCC_X_train.append(imgArr)
#         LCC_y_train.append(label)
#     elif(laterality == "LMLO"):
#         LMLO_X_train.append(imgArr)
#         LMLO_y_train.append(label)
#     elif(laterality == "RCC"):
#         RCC_X_train.append(imgArr)
#         RCC_y_train.append(label)
#     elif(laterality == "RMLO"):
#         RMLO_X_train.append(imgArr)
#         RMLO_y_train.append(label)
#     print(len(X_train)-i, end=" ")

In [32]:
# imgBasePath = "/kaggle/input/rsna-breast-cancer-256-pngs/"
# LCC_X_train = []
# LCC_y_train = []

# LMLO_X_train = []
# LMLO_y_train = []

# RCC_X_train = []
# RCC_y_train = []

# RMLO_X_train = []
# RMLO_y_train = []

In [33]:
# for i in range(len(X_train)):
# #     if(i == 100):
# #       break
    
#     laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#     label = y_train[i]
#     pat_id = X_train._get_value(i, "patient_id")
#     img_id = X_train._get_value(i, "image_id")
#     imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#     #print(i, imgPath)
#     #file = Path(imgPath)
#     img = Image.open(imgPath)
#     #img = img.resize((256, 256))
#     imgArr = np.asarray(img)
#     imgArr = imgArr / 255
#     if(laterality == "LCC"):
#         LCC_X_train.append(imgArr)
#         LCC_y_train.append(label)
#     elif(laterality == "LMLO"):
#         LMLO_X_train.append(imgArr)
#         LMLO_y_train.append(label)
#     elif(laterality == "RCC"):
#         RCC_X_train.append(imgArr)
#         RCC_y_train.append(label)
#     elif(laterality == "RMLO"):
#         RMLO_X_train.append(imgArr)
#         RMLO_y_train.append(label)
#     print(len(X_train)-i, end=" ")

In [34]:
# LCC_X_train = np.array(LCC_X_train)
# LCC_y_train = np.array(LCC_y_train)

# # LMLO_X_train = np.array(LMLO_X_train)
# # LMLO_y_train = np.array(LMLO_y_train)

# # RCC_X_train = np.array(RCC_X_train)
# # RCC_y_train = np.array(RCC_y_train)

# # RMLO_X_train = np.array(RMLO_X_train)
# # RMLO_y_train = np.array(RMLO_y_train)

# print(LCC_X_train.shape, LCC_y_train.shape)
# # print(LMLO_X_train.shape, LMLO_y_train.shape)
# # print(RCC_X_train.shape, RCC_y_train.shape)
# # print(RMLO_X_train.shape, RMLO_y_train.shape)

In [35]:
# for i in range(len(X_train)):
# #     if(i == 100):
# #       break
#     if(not (i in done)):
#         laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#         label = y_train[i]
#         pat_id = X_train._get_value(i, "patient_id")
#         img_id = X_train._get_value(i, "image_id")
#         imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#         #print(i, imgPath)
#         #file = Path(imgPath)
#         img = cv2.imread(imgPath, 0)
#         #img = img.resize((224))
#         imgArr = np.asarray(img, dtype=np.uint16)
#         imgArr = imgArr / 255
#         imgArr = np.reshape(imgArr, (1, 256, 256))
#     #     print(imgArr.nbytes)
#     #     print(sparse.COO(imgArr).nbytes)
#         # print(imgArr.shape)
#         if(laterality == "LMLO"):
#             LMLO_X_train = np.append(LMLO_X_train, imgArr, axis=0)
#             LMLO_y_train = np.append(LMLO_y_train, label)
#             done.append(i)
#     print(len(X_train)-i)

In [36]:
# for i in range(len(X_train)):
# #     if(i == 100):
# #       break
#     if(not (i in done)):
#         laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#         label = y_train[i]
#         pat_id = X_train._get_value(i, "patient_id")
#         img_id = X_train._get_value(i, "image_id")
#         imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#         #print(i, imgPath)
#         #file = Path(imgPath)
#         img = cv2.imread(imgPath, 0)
#         #img = img.resize((224))
#         imgArr = np.asarray(img, dtype=np.uint16)
#         imgArr = imgArr / 255
#         imgArr = np.reshape(imgArr, (1, 256, 256))
#     #     print(imgArr.nbytes)
#     #     print(sparse.COO(imgArr).nbytes)
#         # print(imgArr.shape)
#         if(laterality == "RCC"):
#             RCC_X_train = np.append(RCC_X_train, imgArr, axis=0)
#             RCC_y_train = np.append(RCC_y_train, label)
#             done.append(i)
#     print(len(X_train)-i)

In [37]:
# for i in range(len(X_train)):
# #     if(i == 100):
# #       break
#     if(not (i in done)):
#         laterality = X_train._get_value(i, "laterality") + X_train._get_value(i, "view")
#         label = y_train[i]
#         pat_id = X_train._get_value(i, "patient_id")
#         img_id = X_train._get_value(i, "image_id")
#         imgPath = "{}{}_{}.png".format(imgBasePath, pat_id, img_id)
#         #print(i, imgPath)
#         #file = Path(imgPath)
#         img = cv2.imread(imgPath, 0)
#         #img = img.resize((224))
#         imgArr = np.asarray(img, dtype=np.uint16)
#         imgArr = imgArr / 255
#         imgArr = np.reshape(imgArr, (1, 256, 256))
#     #     print(imgArr.nbytes)
#     #     print(sparse.COO(imgArr).nbytes)
#         # print(imgArr.shape)
#         if(laterality == "RMLO"):
#             RMLO_X_train = np.append(RMLO_X_train, imgArr, axis=0)
#             RMLO_y_train = np.append(RMLO_y_train, label)
#             done.append(i)
#     print(len(X_train)-i)

In [39]:
# rm -rf /kaggle/working/

In [40]:
# np.save("LCCXarr", LCC_X_train)
# np.save("LCCyarr", LCC_y_train)

# np.save("LMLOXarr", LMLO_X_train)
# np.save("LMLOyarr", LMLO_y_train)

# np.save("RCCXarr", RCC_X_train)
# np.save("RCCyarr", RCC_y_train)

# np.save("RMLOXarr", RMLO_X_train)
# np.save("RMLOyarr", RMLO_y_train)

In [41]:
# LMLO_X_train.shape
# print(LMLO_y_train)